In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

train = pd.read_csv('data/digit-recognizer/train.csv')
test = pd.read_csv('data/digit-recognizer/test.csv')

In [8]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Preprocessing

In [9]:
train_data = train.to_numpy()
train_data.shape

(42000, 785)

In [10]:
test_data = test.to_numpy()
test_data.shape

(28000, 784)

In [11]:
#train.describe()

In [12]:
train_data = train_data/255
test_data = test_data/255

In [13]:
X = train.iloc[:35000,1:].to_numpy()
y = train.iloc[:35000,0].to_numpy()
X.shape,y.shape

((35000, 784), (35000,))

In [14]:
test_x = train.iloc[35000:,1:].to_numpy()
test_y = train.iloc[35000:,0].to_numpy()
test_x.shape, test_y.shape

((7000, 784), (7000,))

# Neural Network model

In [30]:
class NeuralNetwork(object):
    def __init__(self):
        self.input_size = 784 # 28x28 pixels are flattened
        self.output_size = 10 # 0-9 classes
        self.hidden_size = 128
        
        self.w1 = np.random.randn(self.input_size,self.hidden_size)
        self.w2 = np.random.randn(self.hidden_size,self.output_size)
    
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def softmax(self, x):
        return np.exp(x)/sum(np.exp(x))
    
    def sigmoidPrime(self,z):
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def forward(self, X):
        self.z2 = np.dot(X, self.w1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.w2)
        y_hat = self.softmax(self.z3)
        
        return y_hat
    
    def costFunction(self, X, y):
        self.y_hat = self.forward(X)
        J = (0.5)*sum((y - self.y_hat)**2)
        return J
    
    def costFunctionPrime(self, X, y):
        self.y_hat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.y_hat), self.sigmoidPrime(self.z3))
        dJdw2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.w2.T)*self.sigmoid(self.z2)
        dJdw1 = np.dot(X.T , delta2)
        return dJdw1, dJdw2
    

In [31]:
nn = NeuralNetwork()

In [38]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories=[range(10)])
y_train = ohe.fit_transform(y.reshape(-1, 1)).toarray()
y_train[:3], y[:3]

(array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), array([1, 0, 1]))

In [35]:
y_hat = nn.forward(X)

/home/pratik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


In [26]:
cost1 = nn.costFunction(X,y_train)

/home/pratik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


In [34]:
#cost1

In [33]:
costprime1 = nn.costFunctionPrime(X,y_train)

/home/pratik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
